In [1]:
from image_reader import ImageReader
from google_scraper import GoogleScraper
from site_predictor import SitePredictor
from similarity_checker import SimilarityChecker
from image_predictor import ImagePredictor
from headline_predictor import HeadlinePredictor
from lingua import Language, LanguageDetectorBuilder
from transformers import pipeline
from textcorrector_v2 import TextCorrectorV2




In [2]:
DIRECTORY = r'C:\Users\cvaal\PycharmProjects\FactAI\200-imgs'
CATEGORIES = ['fake', 'real']
import os

data = []
for category in CATEGORIES:
    datasets = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)
    for img in os.listdir(datasets):
        img_path = os.path.join(datasets, img)
        data.append([img_path, label])

In [3]:
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np

In [4]:
# from PIL import Image
# imgpath = data[53][0]
# img = Image.open(imgpath)
# text = ocr.read_img(imgpath)
# print('uncorrected', text)
# result = happy_tt.generate_text(f'grammar: {text}', args=args)
# print('corrected  ', result.text)
# display(img)

In [5]:

sp = SitePredictor()

ocr = ImageReader()
ip = ImagePredictor()
hp = HeadlinePredictor()
tcV2 = TextCorrectorV2()

languages = [Language.ENGLISH, Language.TAGALOG]
lang_detector = LanguageDetectorBuilder.from_languages(*languages).build()

In [6]:
n = 10
gs = GoogleScraper(n_searches=n)

In [7]:
# s_results = None
# s = ['dssdfjgjdfsjhsfdgksldfglkjsdfg', 'vddffdvdfdfsddsasdasdsadsadsad', 'jskjhdsalkjdsaklsdakjlsad', 'poop']
# tries = 0
# i = 0
# while not s_results and tries < 3:
#     print(tries)
#     tries = tries+1
#     s_results=gs.get_results(s[i])
#     i = i+1


# if tries == 3 and not s_results:
#     print('no searches')


In [7]:
import pickle
checkpoints = r'C:\Users\cvaal\PycharmProjects\FactAI\FULL-SYS-Checkpoint3'
newpath = os.path.join(checkpoints, 'checkpoint39')

with open(os.path.join(newpath,'full_dictionary.pkl'), 'rb') as z:
    full_dict = pickle.load(z)

# # idxval = full_dict['index']

# # hunid = [i+1 for i in range(100)]
# # redo = list(set(hunid) - set(idxval))
# # full_dict.keys()

In [12]:
full_dict

defaultdict(list,
            {'img paths': ['C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\122907073_1819961304846094_3869586409626926890_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\14424888_1083427928372551_4701235641640967098_o-260x146.png',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\176582698_1969395876569302_1330924430200656704_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\186564220_2969838576561218_5282561849880048582_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\186634585_2969838516561224_4718134050357508705_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\197065295_2987031614841914_2821329724372531855_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\200-imgs\\fake\\198462271_2987031284841947_834669912602413752_n.jpg',
              'C:\\Users\\cvaal\\PycharmProjects\\FactAI\\20

In [10]:
import re
def retry_results(text):
    tries = 0
    s_results = None
    while not s_results and tries < 4:
        print(f'retry {tries+1} ')
        if tries == 0:
            print('trying', text)
            s_results=gs.get_results(text)
        
        elif tries == 1:
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results=gs.get_results(text)
        elif tries == 2:
            text = tcV2.summarize(text)
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results=gs.get_results(text)
        elif tries == 3:
            text = tcV2.summarize(text, max_tokens=40)
            text = re.sub(r'[^\w\s]', '', text)
            print('trying:', text)
            s_results=gs.get_results(text)
        

        tries = tries+1

    if tries == 4 and not s_results:
        print('no searches')
        return []
    else:
        print('found searches')
        return s_results, text


In [11]:
# #text = tcV2.correct("""""",'en')
# s_results = retry_results("Rj**sh p522js'' jska ", 'en')
# print(s_results)


In [12]:
sc = SimilarityChecker()

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from collections import defaultdict
imgs = []
texts = []
search_res = []
labels = []
indexes =  []
similarity_vals = []#array of arrays
site_predictions = []#array of arrays
site_predictions_unrounded = []#array of arrays
img_predictions = []
headline_predictions = []
language = []

#full_dict = defaultdict(list)

count = 30

def cut_pad(arr, n):
    arr = np.array(arr, dtype='float32')
    arr = arr[:n] if arr.shape[0] > n else arr
    arr = np.pad(arr, (0, n - arr.shape[0]), mode='constant', constant_values=np.nan)
    return arr

for idx, img_label in enumerate(data[150:]):
    #data_point = data[idx-1]
    r_idx = 150+idx
    ch_idx = r_idx+1
    img = img_label[0]
    label = img_label[1]
    query = ocr.read_img(img)

    print('uncorrected:', query)
    tmp = tcV2.correct(query).replace("\n", "")
    corrected_query = query if len(tmp) < 20 else tmp
    lang_pred = lang_detector.detect_language_of(corrected_query)
    lang = 'en' if str(lang_pred) == 'Language.ENGLISH' else 'tl' if str(lang_pred) == 'Language.TAGALOG' else None
    
    print(r_idx, 'corrected query:', corrected_query, 'label:', label)
    full_res = retry_results(corrected_query)

    if full_res:
        s_results, corrected_query = full_res

        search_title = [title[0] for title in s_results]
        search_links = [title[1] for title in s_results]
        site_preds = sp.predict(search_links)
        
        if site_preds:
            print(site_preds, 'PREDS BEFORE PAD')
            site_preds = cut_pad(np.array(site_preds, dtype='float32'), 10)
            print(site_preds, 'PREDS AFTER PAD')

            full_dict['img paths'].append(img)

            full_dict['raw texts'].append(query)
            full_dict['texts'].append(corrected_query)

            full_dict['index'].append(r_idx)

            full_dict['labels'].append(label)

            full_dict['language'].append(lang)

            sims = None
            
            if lang == 'tl':
                search_title_trunc = [' '.join(headline.split(' ')[:256]) for headline in search_title]
                sims = sc.check_similarity(corrected_query, search_title_trunc, lang)
            else:
                sims = sc.check_similarity(corrected_query, search_title, lang)

            
            sims = cut_pad(sims,10)

            full_dict['similarity vals'].append(sims)

            full_dict['search results'].append(s_results)
            
            full_dict['site predictions'].append(site_preds)


            headline_pred = hp.predict_headlines(corrected_query, lang)
            full_dict['headline predictions'].append(headline_pred)
            
            image_pred = ip.predict_img(img)
            full_dict['image predictions'].append(image_pred)

            

            if ch_idx % 5 == 0:
                count = count + 1
                checkpoints = r'C:\Users\cvaal\PycharmProjects\FactAI\FULL-SYS-Checkpoint3'
                newpath = os.path.join(checkpoints, f'checkpoint{count}')
                if not os.path.exists(newpath):
                    os.makedirs(newpath)

                
                with open(os.path.join(newpath,'full_dictionary.pkl'), 'wb') as z:
                    pickle.dump(full_dict, z)
            
                full_df = pd.DataFrame(full_dict)
                full_df.to_pickle(os.path.join(newpath,'full_df.pkl'))
            
            print(s_results)
            print(site_preds)
            print(sims)

        else:
            continue
    
    else:
        continue
            




text extracted
uncorrected: VIA JIM OWCZARSKI MILWAUKEE JOURNAL SENTINEL BR 0VCa DON'T KNOW MY PURPOSE HERE. JAE CROWDER ON HIS ROLE WITH THE BUCKS
correcting text
150 corrected query: : "I'm just here to do whatever I can to help the team win. I'm here to bring energy, bring toughness, bring a defensive mindset and just do whatever I can to help the team win." label: 1
retry 1 
trying : "I'm just here to do whatever I can to help the team win. I'm here to bring energy, bring toughness, bring a defensive mindset and just do whatever I can to help the team win."
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
found searches
scanning sites: ['https://www.atlassian.com/blog/software-teams/good-teamwork-quotes-youll-like', 'https://everydaypower.com/motivational-sports-quotes/', 'https://positivepsychology.com/mentally-stron

c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[("32 non-corny teamwork quotes you'll actually like - Work Life by Atlassian", 'https://www.atlassian.com/blog/software-teams/good-teamwork-quotes-youll-like'), ('Best Sports Quotes About Greatness & Life | Everyday Power', 'https://everydaypower.com/motivational-sports-quotes/'), ('How to Be Mentally Strong: 14 Ways to Build Mental Toughness', 'https://positivepsychology.com/mentally-strong/'), ('31 Teamwork Quotes That Will Fire Up Your Team! - Bit Blog', 'https://blog.bit.ai/teamwork-quotes/'), ('37 motivational quotes to inspire your employees - Ciphr', 'https://www.ciphr.com/advice/37-motivational-quotes-inspire-team/'), ('Team Building: Games, Ideas, Tips and Techniques – BusinessBalls.com', 'https://www.businessballs.com/team-management/team-building-games-training-ideas-and-tips/'), ('The Leader as Coach', 'https://hbr.org/2019/11/the-leader-as-coach'), ('Summary: How to Win Friends & Influence People', 'https://www.hubspot.com/sales/how-to-win-friends-and-influence-people-sum

c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[('Lawrence Journal-World 02-03-2016 by Lawrence Journal-World - Issuu', 'https://issuu.com/lawrencejournal-world/docs/lawrence_journal-world_02-03-2016'), ('Full text of "Florida Flambeau"', 'https://archive.org/stream/Florida_Flambeau_1992_June/Florida_Flambeau_1992_June_djvu.txt')]
[ 1.  1. nan nan nan nan nan nan nan nan]
[0.11018836 0.15980704        nan        nan        nan        nan
        nan        nan        nan        nan]
text extracted
uncorrected: AIN'T NO FUN WHEK THE RABBIT GOT THE GUNI ANTHONY DAVIS AFTER THE No 7 LAKERS BEAT THE NO. 2 GRIZZLIES, 4-2 IN THE FIRST ROUND TT Qogi 25 [BUZZ ABA 11 6 bbigo
correcting text
152 corrected query: Ain't no fun when the rabbit got the gun, Anthony Davis after the No. 7 Lakers beat the No. 2 Grizzlies, 4-2 in the first round. label: 1
retry 1 
trying Ain't no fun when the rabbit got the gun, Anthony Davis after the No. 7 Lakers beat the No. 2 Grizzlies, 4-2 in the first round.
searching google the first 10 results of google
scra

...

c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[("Andy Rourke, 1964 - 2023: a key thread in The Smiths' rich tapestry", 'https://www.nme.com/en_asia/features/music-features/andy-rourke-the-smiths-bassist-obituary-3445960'), ('The Smiths bassist Andy Rourke dies at 59', 'https://www.rappler.com/entertainment/music/the-smiths-bassist-andy-rourke-dies/'), ('Andy Rourke Dead: The Smiths Bassist Was 59 – Deadline', 'https://deadline.com/2023/05/andy-rourke-dies-the-smiths-bassist-was-59-1235373266/'), ("Andy Rourke Dead: The Smiths' Bassist Was 59 - Variety", 'https://variety.com/2023/artisans/obituaries-people-news/andy-rourke-dead-the-smiths-1235618646/'), ('Smiths Bassist Andy Rourke Tributes: Read The Reactions to His Death – Billboard', 'https://www.billboard.com/music/rock/smiths-bassist-andy-rourke-death-tributes-1235332580/'), ('Andy Rourke, The Smiths bass player, is dead at 59 : NPR', 'https://www.npr.org/2023/05/19/1177038331/andy-rourke-smiths-bass-player-died')]
[ 1.  1.  1.  1.  1.  1. nan nan nan nan]
[0.55543095 0.712868

c:\Users\cvaal\anaconda3\envs\pt-tf-py3.10\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


text extracted
uncorrected: More than half of the worlds large lakes are drying up, study finds A team of international researchers reported that some of the worlds most important freshwater sources from the Caspian Sea between Europe and Asia to South America's Lake Titicaca lost water at a cumulative rate of around 22 gigatonnes per year for nearly three decades. REUTERS PHOTO INQUIRERNET
correcting text


RateLimitError: The server had an error while processing your request. Sorry about that!

In [15]:
search_title_trunc

['Video', 'Video']

In [16]:

checkpoints = r'C:\Users\cvaal\PycharmProjects\FactAI\FULL-SYS-Checkpoint3'
newpath = os.path.join(checkpoints, 'checkpoint20')
# with open(os.path.join(newpath,'full_dictionary.pkl'), 'rb') as z:
#     dic = pickle.load(z)

In [17]:
df = pd.read_pickle(os.path.join(newpath,'full_df.pkl'))

In [18]:
real_hvals = [dic['real'] for dic in df['headline predictions'].values]
fake_hvals = [dic['fake'] for dic in df['headline predictions'].values]

In [19]:
real_ivals = [dic['real'] for dic in df['image predictions'].values]
fake_ivals = [dic['fake'] for dic in df['image predictions'].values]

In [20]:
df['headline real pred'] = real_hvals
df['headline fake pred'] = fake_hvals

df['image real pred'] = real_ivals
df['image fake pred'] = fake_ivals

In [21]:
df.columns

Index(['img paths', 'raw texts', 'texts', 'index', 'labels', 'language',
       'similarity vals', 'search results', 'site predictions',
       'headline predictions', 'image predictions', 'headline real pred',
       'headline fake pred', 'image real pred', 'image fake pred'],
      dtype='object')

In [22]:
df_analysis = df[['raw texts', 'texts', 'labels', 'language',
       'similarity vals', 'site predictions', 'headline real pred', 'headline fake pred', 'image real pred', 'image fake pred']]

In [23]:
df_analysis

,raw texts,texts,labels,language,similarity vals,site predictions,headline real pred,headline fake pred,image real pred,image fake pred
0,"n p ""Idlike to think that thanks to me, women athletes can be themselves. They can play with aggression and pump their fists. They can wear what they want and say what they want and kick butt and be proud of it all. 1 Serena Williams August 9,2022","""""I'd like to think that thanks to me, women athletes can be themselves. They can play with aggression and pump their fists. They can wear what they want and say what they want and kick butt and be proud of it all. - Serena Williams, August 9, 2022.""",0,en,"[0.4954062, 0.61723846, 0.07331531, 0.07772231, 0.5477172, 0.5038338, 0.4738137, 0.46912342, 0.4335133, 0.5255047]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]",9.999957e-01,4.297744e-06,0.985129,0.014871
1,AP Liz Truss promised a shakeup. She was forced out instead. AP Photo/Alberto Pezzali,"""Liz Truss promised a shakeup but was forced out instead.""",0,en,"[0.757781, 0.757781, 0.68414104, 0.7198374, 0.4937871, 0.62154686, 0.52559584, 0.088587806, 0.5272033, 0.08825754]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",9.999996e-01,3.202805e-07,0.722271,0.277729
2,"CANVA FILE PHOTO DRAMA Sinabi ni Senador Jinggoy Estrada na minsan niyang naisip í-ban ang mga foreign-made shows gaya ng mga Korean 74 drama: Ito'y dahil nais ng senador na paigtingin pa ang suporta ng mga Pinoy OBAS sa mga homegrown artists ng Pilipinas, sa WEEKEND gitna ng pagsikat ng foreign-made shows. Ano ang #BosesMo tungkol dito?","""Sinabi ni Senador Jinggoy Estrada na minsan niyang naisip í-ban ang mga foreign-made shows gaya ng mga Korean drama dahil nais niyang paigtingin pa ang suporta ng mga Pinoy sa mga homegrown artists ng Pilipinas sa gitna ng pagsikat ng foreign-made shows. Ano ang #",0,tl,"[0.5207787, 0.7434601, 0.8702723, 0.5310079, 0.77677166, 0.68961406, 0.68544465, 0.5943956, nan, nan]","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, nan, nan, nan]",1.389096e-07,9.999999e-01,0.613181,0.386819
3,Sam Smith and Kim Petras make LGBTQ history on Billboard's Hot 100 chart Sam Smith and Kim Petras became the first openly non-binary solo artist and first openly transgender solo artist; respectively; to ascend to No. 1 on Billboard's Hot 100 song chart. Matt Winkelmeyer /Getty Images for iHeartRadio,Sam Smith and Kim Petras make LGBTQ history on Billboard's Hot 100 chart Sam Smith and Kim Petras became the first openly non-binary solo artist and first openly transgender solo artist; respectively; to ascend to No. 1 on Billboard's Hot 100 song chart. Matt Winkelmeyer /Getty Images for iHeartRadio,0,en,"[0.8441717, 0.8441717, 0.83530045, 0.80543905, 0.55487204, 0.43017864, 0.6586243, 0.8086091, 0.6402322, nan]","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, nan]",1.902389e-07,9.999998e-01,0.833062,0.166938
4,"AP BREAKING NEWS U.S. Senate votes to protect same-sex; interracial marriage Congress is one step closer to passing the landmark bill that would ensure that such unions are enshrined in federal law. Nov. 29,2022","AP BREAKING NEWS U.S. Senate votes to protect same-sex; interracial marriage Congress is one step closer to passing the landmark bill that would ensure that such unions are enshrined in federal law. Nov. 29,2022",0,en,"[0.77322435, 0.7271699, 0.8243269, 0.74752975, 0.8281853, 0.7613627, 0.77938277, 0.8020381, nan, nan]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan, nan, nan]",2.247585e-07,9.999998e-01,0.966953,0.033047
...,...,...,...,...,...,...,...,...,...,...
95,BBM DEVELOPED THE FAMOUS ILOCOS WIND FARMS,"IN THE PHILIPPINES. BBM, A LEADING RENEWABLE ENERGY COMPANY, INSTALLED THE WIND FARMS IN ILOCOS NORTE IN 2008.",1,en,"[0.43800634, 0.48083884, 0.55816007, 0.5484934, 0.37346572, 0.5484934, 0.5918751, 0.0051277895, 0.52059114, 0.56689477]","[0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]",9.999995e-01,4.626908e-07,0.943376,0.056624
96,"K-12 to K-14, plus 2 years mandatory AFP service without pay.

In [36]:
df_analysis.to_pickle('FULL-SYS-RESULTS2.pkl')

In [95]:
import numpy as np
liz = np.array([], dtype='float32')
def cut_pad(arr, n):

    arr = arr[:n] if arr.shape[0] > n else arr
    arr = np.pad(arr, (0, n - arr.shape[0]), mode='constant', constant_values=np.nan)
    return arr

#cut_pad(liz, 10)
if list(liz):
    print('woo')
else:
    print('noo :(')

noo :(


In [ ]:
lang_pred = lang_detector.detect_language_of(text)
lang = 'en' if str(lang_pred) == 'Language.ENGLISH' else 'tl' if str(lang_pred) == 'Language.TAGALOG' else None